## Trigger Conceps

Most particle physics experiments will use a trigger during data acquisition as opposed to streaming out the data continuously. A trigger is a signal that tells the data aquisition system to read out the detector data corresponding to an instance in time (an event) for storage. The latency of a trigger decision is dictated by the buffer size of the system: A trigger decision cannot take longer than the time you can afford to temporarily store the detector data for consecutive events of interests.


### Why does ATLAS need a trigger

The environment at the proton-proton interaction point is extremely busy: We have up to 60 proton interactions per bunch crossings, and 30 million bunch crossings per second!

<img align="center" src="https://s-media-cache-ak0.pinimg.com/originals/3e/cd/07/3ecd07951c28fd36eee321274c82310a.jpg" width="500" />

ATLAS needs a way to be _highly selective_ of the events of interest to keep: Without a trigger system we would need to find a way to record > 40 TB of data per second! 

### What does the trigger select?

"Highly selective" often simply means a requirement of an event to contain reconstructed particles with transverse momentum or energy, $p_T$ or $E_T$, above a certain threshold.

**Why?** This is because the $p_T$/$E_T$ distribution of background particles is exponentially falling:

<img img align="center"  src="https://inspirehep.net/files/7e472b6518d7e64ddb95a6bc7d8fc7db" width="500" />

Meaning:
- Energetic events (in transverse plane) are more interesting because of the lower background rate.
- The $p_T$ threshold is an easy way to controlling trigger rates with intensifying LHC conditions.
    
 > **It does not mean there is no interesting physics below threshold! It is just much more challenging to look there.**

Beside a requirement on the minimum $p_T$, a trigger selection can also require:
- A certain multiplicity of objects, e.g. 4 hadronic jets with $p_T$ > 100 GeV.
- Additional requirements on particle characteristic, e.g. minimum of 2 hadronic jets that have an 85% probability of being initiated by a bottom quark ("b-tagging").
- And even requirements on the event topology, e.g. a requirement of two jets with a high combined invariant mass.

In reality, we have 100s of trigger selections that run parallel, deciding separately what events to keep. 


### The impact of triggers on physics analysis

Beside irrevocably determing which events are stored for an analysis, the decision also impacts the sensitivity of your analysis.
The latter is not only because it will dictate the energy threshold of leading particles in your events for analysis! As already stated, the triggers are required to make a decision within a certain amount of time. This gives them limited amount of time to reconstruct particles to base the decisions on, meaning we will not reconstruct particles as precisely online as we would in stored data. In addition, refinements such as calibration constants and detector alignments can be done later on on offline stored data, which may again deteriote the online reconstruction performance with respect to offline reconstruction performance.

There are therefore two important concepts related to trigger and your physics analysis:
* *Important Trigger Concept 1*: Because of our highly selective trigger, our dataset is heavily biased. When analysing data, the very first step is to choose the trigger that is most appropriate for your physics analysis (the one that has the largest acceptance for your signal) and then filter the data based on whether that trigger fired.
* *Important Trigger Concept 2*: Your selection of physics events will depend on the trigger selection _and_ selection efficiency when the data was taken: Electrons reconstructed more precisely offline may not have had the same measured $p_T$ online, nor have even been correctly identified as an electron.


### Measuring trigger efficiencies

We usually measure the trigger efficiency via "**trigger efficiency turn-on curves**". Turn-on curves simply measure the number of events kept by the trigger/total number of events$^*$ for events containing your signal, as a function of one of the variables used in the trigger selection, usually the $p_T$ of the reconstructed particles.


So let's first get to grips with the concept of trigger turn-ons.


$^*$_By the way, you may ask how do we know the total number of events if they were not triggered by our chosen trigger? What are your ideas?_






# Exercise 1: Emulating a single jet trigger

For the first exercise, we will be emulating a simple jet trigger by generating the momentum spectra of online and offline reconstructed jets. This allows us to study the performance of the trigger in detail: What impacts the trigger performance?

In [ ]:
import numpy
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
#%matplotlib notebook

In [ ]:
# A trigger turn-on emulator for a jet trigger

def get_subplots(number, width, height):
    fig, ax = plt.subplots(1,number)
    fig.set_size_inches(width,height)
    if isinstance(ax, numpy.ndarray):
        axes = ax
    else:
        axes = numpy.array([ax])
    return fig, axes


## First we generate (our exponentially falling pT) spectrum for the offline reconstructed leading jet

col_off = 'darkblue'
col_on = 'darkred'
col_off_kept = 'green'

samples=int(4e6)

offline_jets_pt = 10*numpy.random.exponential(scale=10, size=samples)

print("minimum jet pT generated = %.5f GeV"%(offline_jets_pt.min()))
print("maximum jet pT generated = %.1f GeV"%(offline_jets_pt.max()))

In [ ]:
# --- histogram settings --- 
bin_width = 2 # GeV
h_min = 0
h_max = 600
bins = int((h_max-h_min)/bin_width)


## We then generated the pT spectrum of the online reconstructed leading jets
## Online reconstruction is not the same as offline due to resolution differences! We'll quantify the 
## the difference between online and offline jets via a resolution parameter, sigma.

sigmas = [10.,15.,20.,25] ## Add more for comparisons

fig, axes = get_subplots(len(sigmas), 16, 6)                        

online_jets_pt_array = []

for idx,sigma in enumerate(sigmas):
    gauss = numpy.random.normal(loc=0,scale=sigma, size=samples)
    online_jets_pt = offline_jets_pt + gauss
    online_jets_pt_array.append(online_jets_pt)
    axes[idx].hist(online_jets_pt,bins=bins, range=[h_min,h_max], alpha=0.5, color=col_on, label=f"online, sigma={sigma} GeV")
    axes[idx].hist(offline_jets_pt,bins=bins, range=[h_min,h_max], alpha=0.5, color=col_off, label=f"offline")
    axes[idx].set_ylabel(f"events/{bin_width} GeV")
    axes[idx].set_xlabel("jet pT [GeV]")
    axes[idx].legend(loc="upper right")
plt.show()

In [ ]:
## Now we plot the online vs offline response.
## This should have a spread determined by our sigma set above.
fig, (ax1,ax2) = plt.subplots(1, 2)
fig.set_size_inches(12,4)

colours= mcolors.TABLEAU_COLORS

for idx, online_jets_pt in enumerate(online_jets_pt_array):
    sigma = sigmas[idx]
    pt_ratio = online_jets_pt/offline_jets_pt
    pt_diff = online_jets_pt-offline_jets_pt
    ax1.hist(pt_diff, bins=100, range=[-50,50], alpha=0.2, label=f"$\sigma$={sigma}")
    ax2.hist(pt_ratio, bins=100, range=[-0.5,2.5], alpha=0.4, label=f"$\sigma$={sigma}")
    ax1.set_ylabel(f"count/bin")
    print("standard deviation of distribution = %.1f GeV"%(pt_diff.std()))

ax1.set_xlabel("online jet pT - offline jet pT [GeV]") 
ax2.set_ylabel(f"count/bin")
ax2.set_xlabel("online jet pT/offline jet pT [GeV]")
ax1.legend(loc="upper right")
ax2.legend(loc="upper right")
plt.show()



In [ ]:
## Finally we emulate the trigger selection:

## trigger function
apply_trigger = lambda x, t: x>t  # is x over threshold

## Define our trigger
thresholds = [200]*len(sigmas) # GeV , #thresholds should equal #sigma values

fig, axes = get_subplots(len(sigmas), 16, 6)                        

offline_jets_pt_kept_array = []

for idx, online_jets_pt in enumerate(online_jets_pt_array): 
    sigma = sigmas[idx]
    threshold = thresholds[idx]
    # we check if online jet is above trigger threshold
    triggered_events = apply_trigger(online_jets_pt, threshold)  
    # we get the offline jets for events in which the trigger fired
    offline_jets_pt_kept = offline_jets_pt[triggered_events]
    offline_jets_pt_kept_array.append(offline_jets_pt_kept)
    axes[idx].hist(offline_jets_pt,bins=bins, range=[h_min,h_max], alpha=0.5, color=col_off, label=f"all events")
    axes[idx].hist(offline_jets_pt_kept,bins=bins, range=[h_min,h_max], alpha=0.5, color=col_off_kept, label=f"triggered, thresh={threshold}, $\sigma$={sigma}")
    axes[idx].set_ylabel(f"events/{bin_width} GeV")
    axes[idx].set_xlabel("jet pT [GeV]")
    axes[idx].legend(loc="upper right")
plt.show()

In [ ]:
# Now we plot our trigger turn-on: We plot triggered events/total events per bin. 
fig, ax = plt.subplots()

eff_array=[]

for idx, offline_jets_pt_kept in enumerate(offline_jets_pt_kept_array): 
    sigma=sigmas[idx]
    cnt_total,edges = numpy.histogram(offline_jets_pt, bins=bins, range=[h_min,h_max])
    cnt_kept,edges = numpy.histogram(offline_jets_pt_kept, bins=bins, range=[h_min,h_max])
    eff = cnt_kept/cnt_total
    eff_array.append(eff)
    centered_pt = (edges+bin_width*0.5)[:-1] # recompute bin edges to plot center of pT bin vs efficiency
    ax.plot(centered_pt, eff, marker=".", label=f"$\sigma$={sigma}")

ax.legend(loc="lower right")
plt.xlim(threshold*0.5-max(sigmas), threshold*1.5+max(sigmas))
for threshold in thresholds:
    plt.axvline(threshold,0,1.0, linestyle='--', color='darkred')
plt.axhline(1.0,0,threshold*1.5+max(sigmas),linestyle='--', color='darkorange')


plt.ylabel(f"Trigger efficiency")
plt.xlabel("offline jet pT [GeV]")
plt.show()

### Trigger turn-on analysis

As reference, here is an ATLAS public plot of single jet trigger turn-ons at 420 GeV threshold (with some variations),
<img img align="center" src="https://twiki.cern.ch/twiki/pub/AtlasPublic/JetTriggerPublicResults/eff_PT_j420_data17-data18.png" width="400" />

Our generated plot has the same features as in the ATLAS plot. 
You can see that:

(1) The trigger turn-on is not a step function at threshold: There is a gradual turn on of the trigger efficiency, reaching 100% after the trigger threshold.

(2) We eventually reach 100% trigger efficiency once the offline jet pT is high enough.

**Question**: How does the turn-on change with different sigma values? We can simply add more sigma values to the list above and plot a comparison...


### Measuring jet trigger performance

Analysis using hadronic jets typically ensure that they are working in a region where the trigger is almost 100% efficient, by simply selecting events where the leading offline jet pT is greater than the point where the trigger turn-on curve reaches 99%. This means, events collected below the 99% turn-on point are wasted rate. 

One can measure the performance of the trigger by (1) measuring the jet pT @ 99% turn-on and (2) measuring the fraction of events that are wasted. 
A good trigger will have a 99% turn-on that is close to threshold, and as little as possible wasted rate.

In [ ]:
## Measuring jet trigger performance

total_events = samples

for idx,eff in enumerate(eff_array):
    sigma = sigmas[idx]
    threshold = thresholds[idx]
    offline_jets_pt_kept = offline_jets_pt_kept_array[idx]
    # get first jet pT where trigger efficiency >= 99%
    pt_at_99 = centered_pt[eff>=0.99][0]
    # calculate fraction of events below trigger efficiency below 99%
    total_events_triggered = offline_jets_pt_kept.size
    total_events_for_physics = offline_jets_pt_kept[offline_jets_pt_kept>=pt_at_99].size
    wasted_frac = (total_events_triggered-total_events_for_physics)/total_events_triggered

    print(f"\n--- INFO for trigger: {threshold} GeV threshold, {sigma} GeV sigma ---")
    print("%40s %i GeV "%("99%% trigger efficiency @ jet pT = ",pt_at_99))
    print("%40s %i" %("Number of triggered events = ",total_events_triggered))
    print("%40s %i"%("Number of events for analysis = ", total_events_for_physics))
    print("%40s %.2f"%("fraction of wasted events = ",wasted_frac))
    print("%40s %.1f Hz"%("total rate = ",400*total_events_triggered/total_events)) # arbitrary scaling to get reasonable rate

## Challenge: Calo versus particle flow jets

For the LHC Run 3, the upgraded ATLAS trigger can now reconstruct both calorimeter-based jets and particle flow jets. While calorimeter-based jets are reconstructed from calorimeter topoclusters only, particle flow jets are reconstructed from particle flow objects where calorimeter deposits are linked with tracks to reconstruct a objects with improved momentum measurements and reduced pile-up effects.

Due to their superior precision, particle flow jets are what are reconstructed offline for physics analyses. However, they are expensive objects to reconstruct: Tracking is highly CPU intensive!

For this reason, we cannot afford to reconstruct particle flow jets at the same rate as calo-based jets. On the other hand, the momentum resolution for online particle flow jets is better which means "sharper" trigger turn-ons.

Challenge: Given the following momenta resolutions with respect to offline for online calorimeter jets and online particle flow jets as well as their respective trigger rate constraints, how much higher do you need to adjust the trigger threshold for a pflow jet trigger to be able to afford it? Does it ultimately increase or decrease the physics acceptance for analyses?

<img align="right" src="https://scx1.b-cdn.net/csz/news/800a/2020/1-newprecision.jpg" width="400" />



| type | sigma | max rate|
-------|-------|---------|
| calo jet | 20 | 55 Hz |
| pflow jet | 5 | 40 Hz | 



# Exercise: Measuring the performance of a single lepton trigger 

Next we will do the same thing, but for a single electron trigger and using open data: Specifically a Monte-Carlo sample that contains a decay of a top and an anti-top quark which each decay leptonically (muons or electrons).

In [ ]:
%jsroot on

In [ ]:
import ROOT

base_url = "https://atlas-opendata.web.cern.ch/atlas-opendata/samples/2020/2lep/MC/"
input_file =  "mc_410000.ttbar_lep.2lep.root"
tree_name = "mini" # event "tree" in which information of each event in the data set are defined:
                   # event level information, particles and their properties

# function to retrieve the data from an input file.
#events = uproot.open(f"{base_url}/{input_file}:{tree_name}")
#events.show()

infile = ROOT.TFile.Open(f"{base_url}/{input_file}")
tree = infile.Get("mini")
tree.GetEntries()

In [ ]:
tree.Print()

In [ ]:
canvas = ROOT.TCanvas("Canvas", "c", 800, 600)

# --- histogram settings --- 
bin_width = 5 # GeV
h_min = 0
h_max = 200
bins = int((h_max-h_min)/bin_width)

h_total = ROOT.TH1F("h_total","offline electron pT spectrum; leading offline electron pT [GeV]; events",bins,h_min,h_max)
h_triggered = ROOT.TH1F("h_triggered","triggered offline electron pT spectrum; leading offline electron pT [GeV]; events",bins,h_min,h_max)

h_total.SetLineWidth(2)
h_total.SetLineColor(ROOT.kBlue+2)
h_triggered.SetLineWidth(2)
h_triggered.SetLineColor(ROOT.kOrange+1)

In [ ]:
# Loop through all events: Distinguish between 
# leading electrons from electron triggered events (trigE == True) 
# and leading electrons in all events.

GeV = 1e-3

max_events = 200000

for evt_idx, event in enumerate(tree):
    for l_idx in range(event.lep_n):
        if abs(event.lep_type[l_idx]) == 11:  # 11 is for electron/positron
            electron_pt = event.lep_pt[l_idx]*GeV
            if event.trigE:
                h_triggered.Fill(electron_pt)
            h_total.Fill(electron_pt)
            break # only interested in leading electron
    if evt_idx > max_events: break


In [ ]:
h_total.Draw()
h_triggered.Draw("SAME")
canvas.Draw()

In [ ]:
xmax = 100

## Compute ratio of "triggered" vs "total" electrons/pT bin
eff = ROOT.TEfficiency(h_triggered, h_total)
eff.SetLineWidth(2)


## Draw line to indicate trigger threshold
threshold = 26 # GEV
trig_line = ROOT.TLine(26,0,26,1.0)
trig_line.SetLineColor(ROOT.kRed-2)
trig_line.SetLineWidth(2)
trig_line.SetLineStyle(2)

## Draw line to indicate 100% trigger efficiency
eff_line = ROOT.TLine(0,1.0,xmax,1.0)
eff_line.SetLineColor(ROOT.kOrange+1)
eff_line.SetLineWidth(2)
eff_line.SetLineStyle(2)

eff.Draw()
eff_line.Draw()
trig_line.Draw()
canvas.Draw()

eff.GetPaintedGraph().GetXaxis().SetRangeUser(0,100)
#eff.GetPaintedGraph().GetYaxis().SetRangeUser(0.8,1)
canvas.Draw()

 ### What do we observe?
 
 This matches more or less the officially measured ATLAS trigger efficiency for the 26 GeV single electron trigger:
 
 <img img align="center" src="https://twiki.cern.ch/twiki/pub/AtlasPublic/EgammaTriggerPublicResults/plot_Et_e24_lhvloose_nod0_L1EM20VH_ProbeLooseAndBLayerLLH_d0z0_DataDriven_Rel21_Smooth_vTest_LooseAndBLayerLLH_d0z0_DataDriven_Rel21_Smooth_vTest_LHCC_Sep2017.png" width="500" />
 
 
 **Question**: How does this curve look different to the jet trigger turn-ons? Why is it different?
